# COGS 118A - Project Checkpoint

# Names

- Michael Nodini
- Alex Cagle
- Saransh Malik
- Arthur Hewig
- Maryam Usman

# Abstract 

Our goal for this project is to develop a deep learning / computer vision model to identify breast cancer based on pathology images from the PatchCamelyon (PCam) dataset, using binary classification. The PatchCamelyon dataset consists of 327,680 color images of lymph node samples, each with a size of 96x96 pixels. The images are labeled with one of two classes either indicating the presence or absence of metastatic tissue (which indicates cancerous tissue). We will first pre-process the data by binarizing the pixels to reduce noise. Then, we will be training the last layer of a convolutional neural network on this data. The neural network will be trained to classify the images to a binary label (the detection of cancerous tissue). Success will be measured using sensitivity and specificity. Our main goal is to reduce the false negative rate (by maximizing sensitivity), since false negatives are far more fatal than false positives in this scenario.

# Background

The article Artificial intelligence and computational pathology [Nature], published in 2021 by researchers Miao Cui and David Y. Zhang, discusses the immense potential that artificial intelligence has in the field of clinical pathology. Computational pathology uses technology to improve patient care in pathology and lab medicine. It combines different types of medical data, like images and patient information, to make better diagnoses and treatment plans. However, there are many challenges to overcome, like integrating the data and making sure the technology is used ethically. By utilizing modern computational methods, such as genomics, bioinformatics, and machine learning, and applying them to pathology, computational pathology has the potential to “create personalized diagnosis and treatment plans for patients,” “improve clinical workflow efficiency, diagnostic quality,” and offer a “better-integrated solution to whole-slide images, multi-omics data, and clinical informatics.” Most importantly, computational pathology can “reduce errors in diagnosis and classification.” The article states that machine learning algorithms evaluated by CAMELYON16, which evaluates how well algorithms can detect cancer, “[have] achieved encouraging results with a 92.4% sensitivity in tumor detection rate. In contrast, a pathology could only achieve 73.2% sensitivity.” Artificial intelligence is useful for the task of detecting cancer in pathology slide images because it can be trained to find patterns in data and is able to autonomously learn to solve novel problems. Currently, machine learning methods are being used to assist pathologic diagnosis by looking at cancer cells, cell nuclei, cell divisions, ducts, blood vessels, and more. Deep learning and artificial neural networks (ANNs) resemble human cognition, being composed of nodes (i.e. neurons), which make up an input layer, hidden layers, and an output layer. Convolutional neural networks (CNNs) are particularly adept at handling image classification tasks.

The article Histopathological Cancer Detection with Deep Neural Networks [TowardsDataScience] discusses the use of deep neural networks for histopathological cancer detection. It explains how these advanced algorithms can analyze tissue samples to identify cancer cells with high accuracy. This article highlights the potential of deep neural networks in improving cancer diagnosis and treatment through automated precise detection methods.  

The article The detection of cancer cells in histopathology based on machine vision [ScienceDirect-1], published July 20222 in Computers in Biology and Medicine, discusses why breast cancer detection is so important and how machine vision is able to improve traditional breast cancer detection. Breast cancer is an incredibly pressing issue facing the modern world. It is estimated that ⅛ women will develop breast cancer at some point, and by some metrics, it has overtaken lung leading cause of death out of all cancers worldwide. Furthermore, discovery in its early stages causes the possibility of a successful cure to rise to 80%. Traditional cancer detection through histopathological images is time consuming, difficult, hard to perform on large sets of data, and introduces a large amount of inconsistency from pathologist to pathologist. That’s why we are turning to machine image classification as it has the potential to be more accurate, more consistent, and much more efficient than individual pathologists could ever be.

The article Current applications and challenges of artificial intelligence in pathology [ScienceDirect-2] explores the role of AI and ML in computational pathology. It discusses how AI algorithms can analyze medical images such as histopathology slides to assist in diagnosing diseases. The article emphasizes the potential of AI in improving accuracy, efficiency, and personalized medicine in pathology, while acknowledging the challenges and ethical considerations associated with its implementation. 

[Nature]: https://www.nature.com/articles/s41374-020-00514-0
[TowardsDataScience]: https://towardsdatascience.com/histopathological-cancer-detection-with-deep-neural-networks-3399be879671 
[ScienceDirect-1]: https://www.sciencedirect.com/science/article/pii/S0010482522004280 
[ScienceDirect-2]: https://www.sciencedirect.com/science/article/pii/S2772736X22000081 

# Problem Statement

The problem we are facing is a binary classification problem: determining whether a 96x96 RGBimage is an instance of breast cancer or not. Traditionally this task is determined by a radiologist using their past expertise and training to determine whether someone has breast cancer or not; but we hope to train a  Convolutional Neural Network to make the decision.

# Data

UPDATED FROM PROPOSAL!

You should have obtained and cleaned (if necessary) data you will use for this project.

Please give the following infomration for each dataset you are using
- link/reference to obtain it
- description of the size of the dataset (# of variables, # of observations)
- what an observation consists of
- what some critical variables are, how they are represented
- any special handling, transformations, cleaning, etc you have done should be demonstrated here!

> We are using the PatchCamelyon (PCam) dataset [PCam](https://patchcamelyon.grand-challenge.org/), which was created by Bas Veeling and is used for training and benchmarking machine learning models. It contains 327,680 color images (96x96) of pathology slides with a binary classification that denotes the presence (labeled 1) or absence of cancer (labeled 0). The dataset is in the form of a .h5 file and consists of arrays with the corresponding pixel values (from 0 to 255) for each image.  

> In order to load in the data, we first needed to unzip the files, since they came in a .gz format. Then, we had to read the .h5 files using their corresponding keys. Each .h5 file for the x_train, x_test, and x_valid data as well as the y_train, y_test, and y_valid data were loaded in as numpy arrays. The files with prefix “x” contained the pixel values for each image and the files with prefix “y” contained the binary labels for each image. After loading the files into memory, we were able to use a dataloader to pass the data into our model. We ran into problems on Datahub when trying to load all the files in at once, since we exceeded the storage limits, likely due to the gzipped x_train file and the unzipped x_train file taking up a combined 13.5gb. To counteract this, we unzipped the data locally and then uploaded the .h5 files to Datahub.

> The functions used to unzip the .gz files and read in the .h5 files are included below:


In [ ]:
# Reads in gzipped h5 files
# The key for y_train, y_test, and y_valid is 'y'
# The key for x_train, x_test, and x_valid is 'x'
def load_gz_h5(file_name, key):
    new_file = file_name.rsplit('.', 1)[0]
    with gzip.open(file_name, 'rb') as f_in:
        with open(new_file, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    with h5py.File(new_file, 'r') as h5_file:
        data = h5_file[key][:]
    
    return data

# Files were in the format: 'camelyonpatch_level_2_split_train_x.h5.gz'

# Proposed Solution

Our solution to this problem will be to train a convolutional neural network on the breast cancer image dataset to determine whether a tumor exists in the given image or not. Convolutional neural networks are typically applied to image classification tasks as they are able to learn features from the training images in service of the given task (binary classification). This means we won’t have to manually derive what pixels or groups of pixels in the image are indicative of a positive and negative class, the model will learn through gradient descent. We plan on building our own CNN model using PyTorch / PyTorch Lightning as well as loading a pre-trained ResNet50 model and retraining the last layer to map to our labels. We also plan on using a Naive Bayes model that just predicts the most popular class every time as our benchmark model. Our solution will be tested by splitting our dataset into a training, validation, and test set. The training and validation set will be used during model training with the test set being used to report final evaluation metrics. 

# Evaluation Metrics

An evaluation metric we plan on using is the ROC-AUC. This metric will be useful for measuring performance across different thresholds. Breast cancer diagnosis is not something to be taken lightly so we want to make sure we’re choosing a classification threshold that balances true positive rate and false positive rate appropriately. It will also allow us to directly compare against different models using a single scalar value. We also want to take into account the F-score and accuracy of our models to take into account all aspects of true and false diagnosis.

# Preliminary results

Our preliminary results include using ResNet50, VGG19, and DenseNet on the breast cancer dataset. We used ResNet50 originally but due to it having 48 convolutional layers it shrunk the size of our image down significantly due to it having a base size of 32x32 pixels. Through some online research we found that ResNet doesn’t work too well on images smaller than 224x224. We considered upscaling our dataset but that can mess with line detection convolutions by artificially creating patterns. We decided to try using VGG19 which has 18 convolutions as well as DenseNet which helps with vanishing gradients, strengthens feature propagation, encourages feature reuse, and substantially reduce the number of parameters (https://arxiv.org/abs/1608.06993). 

We experimented with different learning rates varying from 0.0001 to 0.1. We found varying levels of success with some models only increasing accuracy over epochs with the lowest learning rates. This gave us some preliminary knowledge and inspired us to implement a GridSearch in the future to find the optimal learning rate. We also experimented with 
different image sizes, batch sizes, optimizers, and momentum values. Changing the image size can help decrease the amount of noise our model picks up that isn’t useful for predicting whether there is breast cancer in the image or not. Breast cancer only exists within the 32x32 middle pixels so including more information than that could potentially add unneeded noise that our model picks up on for prediction. 

For our preliminary testing we limited the maximum epochs to 10 but plan to create a systematic stopping criterion for our future testing. If the validation loss doesn’t decrease for 2 epochs in a row it’s a sign of overfitting and we’ll stop training the model. Our preliminary performance was 72.38% on Resnet, 77.96% on VGG19, and up to 76.69% on DenseNet. As you can see from the table training attempts were varied depending on our hyperparameters. Hence why we plan on performing a GridSearch on smaller learning rate values.

 ![Models](Models.png)

# Ethics & Privacy

There are a few ethical concerns that always arise using machine learning models. Firstly, the concern of explainability. Since a general practice in medicine is that one must always have full autonomy over their own data, using a machine learning model on that data to check if they have cancer or not can be a huge decision. Because of this, it’s considered good practice to make models explainable, so that users can use informed consent to opt-in or opt-out of certain algorithms being run on their data. Without understanding the ways that machine learning models work, people may not be able to give full informed consent to use their data, which brings up ethical concerns of the deployment of this model into the healthcare industry. 

Another common concern in reinforcement learning models (that this may be in deployment) is that of split consent. One may consent to use their data as an algorithm subject (such that their data is run through the model to get a response), but not as a data subject (such that their data is used to retrain the model). Especially in a field like medicine where biological records are considered PII data, having full control over such data is important. Further, with data consent laws in the state of California requiring the ability to revoke access to any of your data, it’s a question of how this model may be trained such that data is used in a manner that includes informed consent.

# Team Expectations 

1. Team Expectation 1: Showing up to scheduled group meetings (on time).
2. Team Expectation 2: Putting in a roughly equal amount of effort on assignments.
3. Team Expectation 3: Communicating clearly when something is not going well in a specific section of the project, so that we can work together to rectify it in a timely manner

How will we handle conflict and difficulty: Communicate. If something isn’t going well, we can talk through it together to come up with solutions!

How we will handle making decisions and setting goals/schedule: We have our weekly Tuesday meeting scheduled. We make all major decisions together during this meeting.

How much work we expect from each other: The amount of work we decide together in the meeting. As long as you get done what you say, that’s perfect.

How we will handle deadlines: We meet every tuesday, so we discuss what needs to be done by the deadline in that meeting. 

# Project Timeline Proposal

![image](timeline.png)

# Footnotes
<a name="lorenznote"></a>1.[^](#lorenz): Lorenz, T. (9 Dec 2021) Birds Aren’t Real, or Are They? Inside a Gen Z Conspiracy Theory. *The New York Times*. https://www.nytimes.com/2021/12/09/technology/birds-arent-real-gen-z-misinformation.html<br> 
<a name="admonishnote"></a>2.[^](#admonish): Also refs should be important to the background, not some randomly chosen vaguely related stuff. Include a web link if possible in refs as above.<br>
<a name="sotanote"></a>3.[^](#sota): Perhaps the current state of the art solution such as you see on [Papers with code](https://paperswithcode.com/sota). Or maybe not SOTA, but rather a standard textbook/Kaggle solution to this kind of problem
